In [3]:
#import dependencies
import pandas as pd
import gmaps
import requests
import ipywidgets as widgets

#Import API key
from config import g_key

In [4]:
#Store the saved CSV created in part one into a data frame
city_data_df = pd.read_csv('weather_data/cities.csv')
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Sakakah,SA,2020-11-20 06:55:37,29.97,40.21,59.00,54,40,3.36
1,1,Inongo,CD,2020-11-20 06:59:06,-1.95,18.27,73.62,91,100,3.33
2,2,Hermanus,ZA,2020-11-20 06:59:07,-34.42,19.23,69.01,62,0,11.99
3,3,Souillac,MU,2020-11-20 06:59:07,-20.52,57.52,73.40,88,75,3.36
4,4,Dingle,PH,2020-11-20 06:59:07,11.00,122.67,85.12,77,99,9.15


In [5]:
#Get the data types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [6]:
#Configure gmaps to use your Google API key
gmaps.configure(api_key = g_key)

## Create a Maximum Temperature Heatmap

In [7]:
#Get the maximum temperature
max_temp = city_data_df['Max Temp']
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
#Get the latitude and longitude
locations = city_data_df[['Lat', 'Lng']]

#Assign the figure variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], 
                                 dissipating=False, max_intensity=300, point_radius=4)

#Add the heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Humidity Heatmap

In [9]:
locations = city_data_df[['Lat', 'Lng']]
humidity = city_data_df['Humidity']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Percent Cloudiness Heatmap

In [10]:
locations = city_data_df[['Lat', 'Lng']]
clouds = city_data_df['Cloudiness']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Create a Wind Speed Heatmap

In [11]:
locations = city_data_df[['Lat', 'Lng']]
wind = city_data_df['Wind Speed']
fig = gmaps.figure(center=(30.0, 31), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, 
                                 dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

## Vacation Criterias

In [12]:
#Ask the customer to add a minimum and maximum temperature value
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [13]:
#Filter the data set to find the cities that fit the crieteria
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
4,4,Dingle,PH,2020-11-20 06:59:07,11.00,122.67,85.12,77,99,9.15
11,11,Kapaa,US,2020-11-20 06:59:09,22.08,-159.32,77.00,83,20,13.87
12,12,Hithadhoo,MV,2020-11-20 06:59:09,-0.60,73.08,81.59,69,100,5.68
19,19,Busselton,AU,2020-11-20 06:54:17,-33.65,115.33,75.99,58,58,12.66
23,23,Vila Velha,BR,2020-11-20 06:59:12,-20.33,-40.29,77.00,83,75,16.11
27,27,Port Macquarie,AU,2020-11-20 06:54:26,-31.43,152.92,79.00,60,0,1.01
34,34,Kimbe,PG,2020-11-20 06:59:14,-5.55,150.14,78.69,88,100,6.51
36,36,Atuona,PF,2020-11-20 06:59:14,-9.80,-139.03,78.28,73,7,6.85
40,40,Chirongui,YT,2020-11-20 06:59:16,-12.93,45.15,84.20,70,75,10.29
43,43,Georgetown,MY,2020-11-20 06:59:16,5.41,100.34,87.80,70,20,6.93


In [14]:
preferred_cities_df.count()

City_ID       173
City          173
Country       169
Date          173
Lat           173
Lng           173
Max Temp      173
Humidity      173
Cloudiness    173
Wind Speed    173
dtype: int64

In [15]:
#Create a data frame called hotel_df to store hotel names along with city, country, max temp, and coordinates
hotel_df = preferred_cities_df[['City', 'Country', 'Max Temp', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Dingle,PH,85.12,11.00,122.67,
11,Kapaa,US,77.00,22.08,-159.32,
12,Hithadhoo,MV,81.59,-0.60,73.08,
19,Busselton,AU,75.99,-33.65,115.33,
23,Vila Velha,BR,77.00,-20.33,-40.29,
27,Port Macquarie,AU,79.00,-31.43,152.92,
34,Kimbe,PG,78.69,-5.55,150.14,
36,Atuona,PF,78.28,-9.80,-139.03,
40,Chirongui,YT,84.20,-12.93,45.15,
43,Georgetown,MY,87.80,5.41,100.34,


In [16]:
#Set parameters to search for a hotel
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}

#Iterate through the data frame
for index, row in hotel_df.iterrows():
    #Get the latitude and longitude
    lat = row['Lat']
    lng = row['Lng']
    
    #Add the latitude and longitude to location key for the params dictionary
    params['location'] = f"{lat},{lng}"
    
    #Use the search term: "lodging" and our latitude and longitude
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    #Make request and get the JSON data from the search
    hotels = requests.get(base_url, params=params).json()
    #Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        print('Hotel not found... skipping.')

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Dingle,PH,85.12,11.00,122.67,SEGAYA PROPERTY
11,Kapaa,US,77.00,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
12,Hithadhoo,MV,81.59,-0.60,73.08,Scoop Guest House
19,Busselton,AU,75.99,-33.65,115.33,Observatory Guest House
23,Vila Velha,BR,77.00,-20.33,-40.29,Hotel Vitória Palace
27,Port Macquarie,AU,79.00,-31.43,152.92,Rydges Port Macquarie
34,Kimbe,PG,78.69,-5.55,150.14,Kimbe Bay Hotel
36,Atuona,PF,78.28,-9.80,-139.03,Villa Enata
40,Chirongui,YT,84.20,-12.93,45.15,La Villa Mahatsara
43,Georgetown,MY,87.80,5.41,100.34,Cititel Penang


In [20]:
#Add a heatmap of temperature for vacation spots
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

#Store the data frame row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

 ### Code Breakdown ###
 #Set hotel_info equal to the info_box_content
 #In the list comprehension, each "row" of hotel_df is iteratted and then formatted into the template
 #Only rows defined in the info_box_template are used, which are hotel name, city, country, max temp

In [22]:
#Add a heatmap of temperature for vacation spots and a pop-up marker for each city
locations = hotel_df[['Lat', 'Lng']]
max_temp = hotel_df['Max Temp']
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, 
                                 max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))